# Hourly SSH computation using QGSW model initialized with NATL60

----------------------------------------------------------------------------------------------------

## Software version 

In [1]:
import sys
print (sys.version)

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 12:04:33) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


## Library importation 

In [2]:
%matplotlib inline
import sys,os,shutil
#sys.path.insert(0,'/Users/sammymetref/Documents/Boost-Swot/Notebooks/GitHub/Personal_Files/2018/Scripts/2018-03-15-sm-qgsw-DI-master-modified/') 
import numpy as np
import matplotlib.pylab as plt
import time
import netCDF4 as nc
import qgsw
import modgrid

## Initialization 

In [3]:
file='/Users/sammymetref/Documents/Boost-Swot/Notebooks/GitHub/Personal_Files/2018/Data/OSMOSIS/NATL60OSMO-CJM165_y2012m06d14-y2013m10d01.1d_SSHdegrad.nc'
fid = nc.Dataset(file)
lon2d=np.array(fid.variables["nav_lon"][:])
lat2d=np.array(fid.variables["nav_lat"][:])
SSH=np.array(fid.variables["degraded_sossheig"][:,:])

## Parameter settings 

In [4]:

SSH_t0 = SSH[0,:,:]# Set constant Rossby first baroclinic phase speed to constant value
c=SSH_t0*0. + 2.5 # in m/s

tint=int(86400/24*3) # Time of propagator integration in seconds. Can be positive or negative
deltat=int(86400/24) # Time period of outputs
dt=600 # propagator time step

## Documentation on the QGSW operator

In [5]:
help(qgsw)

Help on module qgsw:

NAME
    qgsw

FUNCTIONS
    cos(...)
        cos(x)
        
        Return the cosine of x (measured in radians).
    
    isnan(...)
        isnan(x) -> bool
        
        Return True if x is a NaN (not a number), and False otherwise.
    
    qgsw(Hi=None, c=None, lon=None, lat=None, tint=None, dtout=None, dt=None, obsspace=None, Hm=None, rappel=None, snu=None)
        QG Shallow Water model
        
        Args:
            Hi (2D array): Initial SSH field.
            c (same size as Hi): Rossby first baroclinic phase speed
            lon (2D array): longitudes
            lat (2D array): latitudes
            tint (scalar): Time of propagator integration in seconds. Can be positive (forward integration) or negative (backward integration)
            dtout (scalar): Time period of outputs
            dt (scalar): Propagator time step
        
        Returns:
            SSH: 3D array with dimensions (timesteps, height, width), SSH forecast
    
    sin

## Applying the QGSW operator

In [6]:
Hf,trash=qgsw.qgsw(Hi=SSH_t0, c=c, lon=lon2d, lat=lat2d, tint=tint, dtout=deltat, dt=dt,rappel=None,snu=0.)

Hf.shape

(4, 301, 175)

## Saving SSH outputs

In [7]:
from datetime import datetime
from datetime import timedelta
date0=datetime(2012,6,14,0)
deltatoutput=timedelta(hours=1)
for it in range(np.shape(Hf)[0]):
    print(date0)
    path='/Users/sammymetref/Documents/Boost-Swot/Notebooks/GitHub/Personal_Files/2018/Data/OSMOSIS/QGSWOSMO-IniNATL60_1h/'
    str(it)
    ncout = nc.Dataset(path+'QGSWOSMO-IniNATL60_y'+str(date0.year)+'m'+str(date0.month)+'d'+str(date0.day)+'h'+str(date0.hour)+'_SSHdegrad.nc', 'w', format='NETCDF3_CLASSIC')
    ncout.createDimension('x', lon2d.shape[0])
    ncout.createDimension('y', lat2d.shape[1])
    ncout.createDimension('time', None)
    nclon = ncout.createVariable('lon', 'f', ('x',))
    nclat = ncout.createVariable('lat', 'f', ('y',))
    nctim = ncout.createVariable('time', 'f', ('time',))
#   nchei = ncout.createVariable('SSH', 'f', ('time','y','x',))
    nchei = ncout.createVariable('SSH', 'f', ('y','x',)) 
    nclat[:] = lat2d[0,:]
    nclon[:] = lon2d[:,0] 
#nctim[:]=range(0,tint+deltat,deltat)
    nctim[:]=deltat*it
#nchei[:,:,:] = Hf[:,:,:]
    nchei[:,:] = Hf[it,:,:]
    ncout.close()
    date0=date0+deltatoutput

2012-06-14 00:00:00
2012-06-14 01:00:00
2012-06-14 02:00:00
2012-06-14 03:00:00
